## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [1]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [2]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [3]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [4]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [5]:
df=pd.read_csv("2021-01-11.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [6]:
end=tdf.index[2].date()

In [7]:
trading_days=250#len(df)/number_of_years(tdf) 

In [8]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [9]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.018784,0.000197,0.020427,0.006319,0.014751,0.010515,0.033349,0.010874,-0.017222,0.009435,...,0.009872,0.032986,0.011799,0.029049,0.014480,-0.013351,0.056647,0.004052,0.035903,0.057652
2,0.001617,-0.019235,-0.020225,0.010517,-0.009523,0.000000,0.060775,0.013830,-0.013050,0.015850,...,0.004654,0.000000,-0.018850,0.017799,0.031909,-0.024539,0.025212,0.009880,0.005714,0.038587


In [10]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [11]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,-0.017222,0.0,...,0.0,0.0,0.00000,0.0,0.0,-0.013351,0.0,0.0,0.0,0.0
2,0.0,-0.019235,-0.020225,0.0,-0.009523,0.0,0.0,0.0,-0.013050,0.0,...,0.0,0.0,-0.01885,0.0,0.0,-0.024539,0.0,0.0,0.0,0.0


In [12]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [13]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.036839,0.041697,0.087236,-0.009008,0.052090,0.022563,-0.058853,-0.006344,-0.008954,-0.013766,...,0.011196,0.070785,0.065769,0.024141,-0.037401,0.024009,0.067457,-0.012505,0.064782,0.040912
BAJFINANCE,0.041697,0.047197,0.098742,-0.010196,0.058960,0.025539,-0.066616,-0.007181,-0.010134,-0.015582,...,0.012673,0.080121,0.074444,0.027325,-0.042334,0.027175,0.076355,-0.014155,0.073326,0.046308
BAJAJFINSV,0.087236,0.098742,0.206580,-0.021332,0.123351,0.053431,-0.139368,-0.015023,-0.021202,-0.032598,...,0.026514,0.167623,0.155745,0.057166,-0.088568,0.056854,0.159743,-0.029614,0.153406,0.096883
BRITANNIA,-0.009008,-0.010196,-0.021332,0.002203,-0.012738,-0.005517,0.014392,0.001551,0.002189,0.003366,...,-0.002738,-0.017309,-0.016083,-0.005903,0.009146,-0.005871,-0.016496,0.003058,-0.015841,-0.010004
DIVISLAB,0.052090,0.058960,0.123351,-0.012738,0.073654,0.031904,-0.083218,-0.008970,-0.012660,-0.019465,...,0.015832,0.100090,0.092997,0.034135,-0.052885,0.033948,0.095384,-0.017683,0.091601,0.057850
GRASIM,0.022563,0.025539,0.053431,-0.005517,0.031904,0.013820,-0.036047,-0.003886,-0.005484,-0.008431,...,0.006858,0.043355,0.040283,0.014786,-0.022908,0.014705,0.041317,-0.007659,0.039678,0.025058
HCLTECH,-0.058853,-0.066616,-0.139368,0.014392,-0.083218,-0.036047,0.094024,0.010135,0.014304,0.021992,...,-0.017888,-0.113086,-0.105073,-0.038567,0.059752,-0.038356,-0.107770,0.019979,-0.103495,-0.065362
HDFCBANK,-0.006344,-0.007181,-0.015023,0.001551,-0.008970,-0.003886,0.010135,0.001092,0.001542,0.002371,...,-0.001928,-0.012190,-0.011326,-0.004157,0.006441,-0.004134,-0.011617,0.002154,-0.011156,-0.007045
HINDALCO,-0.008954,-0.010134,-0.021202,0.002189,-0.012660,-0.005484,0.014304,0.001542,0.002176,0.003346,...,-0.002721,-0.017204,-0.015985,-0.005867,0.009090,-0.005835,-0.016395,0.003039,-0.015745,-0.009944
HINDUNILVR,-0.013766,-0.015582,-0.032598,0.003366,-0.019465,-0.008431,0.021992,0.002371,0.003346,0.005144,...,-0.004184,-0.026451,-0.024577,-0.009021,0.013976,-0.008972,-0.025208,0.004673,-0.024208,-0.015288


In [14]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [15]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [16]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [17]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [18]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,1.796007,0.10903,16.144242,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [19]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [20]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [21]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array(BALANCE([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
]))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [22]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,1.796007,0.059863,29.404036,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [23]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')